# Data cleaning for scripts 01_⌚_StopTimesPerRoute.py and 02_📍_BusLocator.py

### Download Required Files

Please head over to the following link: [GTFS Dublin Bus Data](https://www.transportforireland.ie/transitData/Data/GTFS_Dublin_Bus.zip).  
This will download the file `GTFS_Dublin_Bus.zip`. After downloading, extract the files.

We will need the following files from the extracted data:

1. `routes.txt`
2. `stops.txt`
3. `stop_times.txt`
4. `trips.txt`

These files will be used to generate the following files required for our app:

1. `Routes.txt`
2. `StopMapLocation.txt`
3. `StopTimesPerTrip.txt`
4. `Towards.txt`

### Important: As of today, February 8, 2025, the NTA has changed the numbering for route_id and other attributes. If the app breaks, this is most likely the cause.

Cleaning routes.txt file for script 01_⌚_StopTimesPerRoute.py and 02_📍_BusLocator.py

In [1]:
#Importing required libraries:
import pandas as pd
import os

In [2]:
# Define the relative path to the target directory
output_dir = "../data"

# Ensure the target directory exists
os.makedirs(output_dir, exist_ok=True)

## CleanedRoutes.txt

In [3]:
# Read the routes.txt file
file_path = "routes.txt" 
routes_df = pd.read_csv(file_path)

# Explicitly create a new DataFrame with only the required columns
cleaned_routes = routes_df[["route_id", "route_short_name", "route_long_name"]].copy()

# Create a new column for the concatenated values
cleaned_routes["route_concat"] = cleaned_routes["route_short_name"].astype(str) + ". " + cleaned_routes["route_long_name"]

# Define a custom sorting key function
def custom_sort_key(route_short_name):
    import re
    match = re.match(r'(\d+)([A-Za-z]*)', route_short_name)
    if match:
        num_part = int(match.group(1))
        alpha_part = match.group(2)
        return (num_part, alpha_part or "")
    return (float('inf'), route_short_name)

# Apply the custom sorting
cleaned_routes["sort_key"] = cleaned_routes["route_short_name"].apply(custom_sort_key)
cleaned_routes = cleaned_routes.sort_values(by="sort_key").drop(columns=["sort_key"])
# Reset the index
cleaned_routes = cleaned_routes.reset_index(drop=True)
cleaned_routes.head(20)

,route_id,route_short_name,route_long_name,route_concat
0,4486_88252,1,Sandymount - O'Connell Street - Santry,1. Sandymount - O'Connell Street - Santry
1,4486_88177,4,Monkstown Avenue - Harristown,4. Monkstown Avenue - Harristown
2,4486_88174,6,Howth Dart Stn - Lower Abbey Street,6. Howth Dart Stn - Lower Abbey Street
3,4486_88218,7,Brides Glen Luas - Mountjoy Square,7. Brides Glen Luas - Mountjoy Square
4,4486_88219,7A,Loughlinstown - Mountjoy Sq,7A. Loughlinstown - Mountjoy Sq
5,4486_88220,7B,Shankill - Mountjoy Square,7B. Shankill - Mountjoy Square
6,4486_88221,7D,Dalkey - Mountjoy Square,7D. Dalkey - Mountjoy Square
7,4486_88222,7E,Mountjoy Square - Dalkey,7E. Mountjoy Square - Dalkey
8,4486_88247,9,Limekiln Avenue - Charlestown,9. Limekiln Avenue - Charlestown
9,4492_88176,11,Phoenix Park - Sandyford Business District,11. Phoenix Park - Sandyford Business District


In [4]:
# Explicitly create a new DataFrame with only the required columns
cleaned_routes = cleaned_routes[["route_id", "route_short_name", "route_concat"]].copy()

cleaned_routes.rename(columns={"route_short_name": "incoming_bus","route_concat": "route_name"}, inplace=True)

cleaned_routes

,route_id,incoming_bus,route_name
0,4486_88252,1,1. Sandymount - O'Connell Street - Santry
1,4486_88177,4,4. Monkstown Avenue - Harristown
2,4486_88174,6,6. Howth Dart Stn - Lower Abbey Street
3,4486_88218,7,7. Brides Glen Luas - Mountjoy Square
4,4486_88219,7A,7A. Loughlinstown - Mountjoy Sq
...,...,...,...
115,4486_88261,X27,X27. Salesian College - Aghards Rd - UCD
116,4486_88262,X28,X28. Salesian College - Celbridge Main St - UCD
117,4486_88263,X30,X30. Dodsboro - UCD
118,4486_88264,X31,X31. River Forest - Earlsfort Terrace


In [5]:
# This file will be used in 01_⌚_StopTimesPerRoute.py and 02_📍_BusLocator.py
# Construct the full path for the output file
output_file_path = os.path.join(output_dir, "Routes.txt")

# Optionally save the updated data to a new file
cleaned_routes.to_csv(output_file_path, index=False)

Data cleaning for 01_⌚_StopTimesPerRoute.py.

## Towards.txt

In [6]:
# Read the trips.txt file
file_path_1 = "trips.txt"  # Replace with the actual path if needed
trips_df = pd.read_csv(file_path_1)
trips_df.head(30)

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id
0,4486_88211,162,4486_10216,Abbey St,2083,1,4486_7778195_5041033,4486_313
1,4486_88213,162,4486_10333,Portmarnock,4873,0,4486_7778195_6042026,4486_325
2,4486_88213,162,4486_10337,Portmarnock,4865,0,4486_7778195_6042024,4486_326
3,4486_88213,162,4486_10443,Talbot Street,4855,1,4486_7778195_6042027,4486_328
4,4486_88216,162,4486_10636,Enniskerry,2280,0,4486_7778195_1044005,4486_340
5,4486_88223,162,4486_10750,Belarmine,117,0,4486_7778195_2047005,4486_354
6,4486_88223,162,4486_10803,Poolbeg St,139,1,4486_7778195_2047001,4486_361
7,4486_88224,162,4486_10881,The Square,5742,0,4486_7778195_3049010,4486_362
8,4486_88224,162,4486_10962,Pearse St,5724,1,4486_7778195_3049009,4486_367
9,4486_88226,162,4486_11069,Ringsend Road,412,1,4486_7778195_3423017,4486_374


In [7]:
trips_df

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id
0,4486_88211,162,4486_10216,Abbey St,2083,1,4486_7778195_5041033,4486_313
1,4486_88213,162,4486_10333,Portmarnock,4873,0,4486_7778195_6042026,4486_325
2,4486_88213,162,4486_10337,Portmarnock,4865,0,4486_7778195_6042024,4486_326
3,4486_88213,162,4486_10443,Talbot Street,4855,1,4486_7778195_6042027,4486_328
4,4486_88216,162,4486_10636,Enniskerry,2280,0,4486_7778195_1044005,4486_340
...,...,...,...,...,...,...,...,...
20371,4486_88211,340,4492_9995,Swords Manor,1882,0,4492_7778195_5041012,4492_309
20372,4486_88211,342,4492_9996,Swords Manor,16447,0,4492_7778195_5041006,4492_309
20373,4486_88211,341,4492_9997,Swords Manor,9904,0,4492_7778195_5041007,4492_310
20374,4486_88211,340,4492_9998,Swords Manor,1760,0,4492_7778195_5041018,4492_309


In [8]:
# Keep only the first occurrence of each distinct combination of route_id, trip_headsign, and direction_id
SelectDirection_df = trips_df[['route_id', 'trip_id', 'trip_headsign', 'direction_id']] \
    .drop_duplicates(subset=['route_id', 'trip_headsign', 'direction_id'], keep='first') \
    .sort_values(by='route_id')

SelectDirection_df
SelectDirection_df1 = SelectDirection_df.copy()

In [9]:
SelectDirection_df1=SelectDirection_df.copy()
SelectDirection_df

,route_id,trip_id,trip_headsign,direction_id
15,4486_88174,4486_11484,Abbey St Lower,1
14,4486_88174,4486_11430,Howth Station,0
15978,4486_88175,4492_604,Phoenix Pk,1
15566,4486_88175,4492_567,Donnybrook Stadium,0
242,4486_88176,4486_563,Parnell Square,1
...,...,...,...,...
10327,4486_88292,4492_19013,River Forest,1
13078,4492_88176,4492_343,Sandyford B.D.,0
14289,4492_88176,4492_452,O'Connell St Lower,0
14322,4492_88176,4492_455,Phoenix Pk,1


In [10]:
display = SelectDirection_df[(SelectDirection_df["route_id"] == "4486_88181")]
display

,route_id,trip_id,trip_headsign,direction_id
14472,4486_88181,4492_4685,Ballinteer,0
14721,4486_88181,4492_4909,Dublin Airport,1
231,4486_88181,4486_4908,O'Connell Street,0
14974,4486_88181,4492_5136,O'Connell Street,1


In [11]:
# Construct the full path for the output file
output_file_path1 = os.path.join(output_dir, "Towards.txt")

# Save the updated data to a new file
SelectDirection_df.to_csv(output_file_path1, index=False)  #Print this file into assets/data

## StopTimesPerTrip.txt

In [12]:
# Read the Routes.txt file
file_path_2 = "../data/Routes.txt"  
routes_df = pd.read_csv(file_path_2)

routes_df.head(100)

,route_id,incoming_bus,route_name
0,4486_88252,1,1. Sandymount - O'Connell Street - Santry
1,4486_88177,4,4. Monkstown Avenue - Harristown
2,4486_88174,6,6. Howth Dart Stn - Lower Abbey Street
3,4486_88218,7,7. Brides Glen Luas - Mountjoy Square
4,4486_88219,7A,7A. Loughlinstown - Mountjoy Sq
...,...,...,...
95,4486_88273,E2,E2. Dun Laoghaire - Harristown
96,4486_88274,G1,G1. Red Cow LUAS - Spencer Dock
97,4486_88275,G2,G2. Liffey Valley SC - Spencer Dock
98,4486_88276,H1,H1. Baldoyle - Lower Abbey Street


In [13]:
# Merge route_name into trips_df based on route_id
trips_df = trips_df.merge(routes_df[["route_id", "route_name","incoming_bus"]], on='route_id', how='left')
# Organising trips_df
trips_df = trips_df[["trip_id","route_id","incoming_bus","route_name","trip_headsign","direction_id"]].copy()

trips_df.head(30)

,trip_id,route_id,incoming_bus,route_name,trip_headsign,direction_id
0,4486_10216,4486_88211,41C,41C. Swords Manor Via River Valley - Lower Abb...,Abbey St,1
1,4486_10333,4486_88213,42,42. Sand's Hotel Portmarnock - Eden Quay,Portmarnock,0
2,4486_10337,4486_88213,42,42. Sand's Hotel Portmarnock - Eden Quay,Portmarnock,0
3,4486_10443,4486_88213,42,42. Sand's Hotel Portmarnock - Eden Quay,Talbot Street,1
4,4486_10636,4486_88216,44,44. Enniskerry - DCU,Enniskerry,0
5,4486_10750,4486_88223,47,47. Poolbeg Street - Belarmine,Belarmine,0
6,4486_10803,4486_88223,47,47. Poolbeg Street - Belarmine,Poolbeg St,1
7,4486_10881,4486_88224,49,49. Tallaght (The Square) - Pearse St,The Square,0
8,4486_10962,4486_88224,49,49. Tallaght (The Square) - Pearse St,Pearse St,1
9,4486_11069,4486_88226,52,52. Intel - Ringsend,Ringsend Road,1


In [14]:
# Read the stop_times.txt file
file_path_3 = "stop_times.txt"  
stop_times_df = pd.read_csv(file_path_3)
stop_times_df1 = stop_times_df.copy()
stop_times_df.head(100)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint
0,4486_10216,23:15:00,23:15:00,8240DB004843,1,Abbey St,0,1,1
1,4486_10216,23:15:47,23:15:47,8240DB004905,2,NaN,0,0,0
2,4486_10216,23:16:33,23:16:33,8240DB004906,3,NaN,0,0,0
3,4486_10216,23:17:06,23:17:06,8240DB004907,4,NaN,0,0,0
4,4486_10216,23:17:27,23:17:27,8240DB004908,5,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...
95,4486_10333,23:46:41,23:46:41,8240DB003574,37,NaN,0,0,0
96,4486_10333,23:48:08,23:48:08,8240DB003641,38,NaN,0,0,1
97,4486_10333,23:48:48,23:48:48,8240DB003642,39,NaN,0,0,0
98,4486_10333,23:51:03,23:51:03,8240DB003643,40,NaN,0,0,0


In [15]:
StopTimesPerTrip_df = stop_times_df[["trip_id","stop_id","arrival_time"]].copy()
StopTimesPerTrip_df

,trip_id,stop_id,arrival_time
0,4486_10216,8240DB004843,23:15:00
1,4486_10216,8240DB004905,23:15:47
2,4486_10216,8240DB004906,23:16:33
3,4486_10216,8240DB004907,23:17:06
4,4486_10216,8240DB004908,23:17:27
...,...,...,...
1076100,4492_9999,8240DB004912,10:33:42
1076101,4492_9999,8240DB004913,10:34:24
1076102,4492_9999,8240DB004914,10:34:53
1076103,4492_9999,8240DB004915,10:35:29


In [16]:
StopTimesPerTrip_df.dtypes

trip_id         object
stop_id         object
arrival_time    object
dtype: object

In [17]:
# Construct the full path for the output file
output_file_path3 = os.path.join(output_dir, "StopTimesPerTrip.txt")

# Saving stop_times_per_route_df into StopTimesPerRoute.txt
StopTimesPerTrip_df.to_csv(output_file_path3, index=False) #Print this file into assets/data

## StopMapLocation.txt

In [18]:
SelectDirection_df1

,route_id,trip_id,trip_headsign,direction_id
15,4486_88174,4486_11484,Abbey St Lower,1
14,4486_88174,4486_11430,Howth Station,0
15978,4486_88175,4492_604,Phoenix Pk,1
15566,4486_88175,4492_567,Donnybrook Stadium,0
242,4486_88176,4486_563,Parnell Square,1
...,...,...,...,...
10327,4486_88292,4492_19013,River Forest,1
13078,4492_88176,4492_343,Sandyford B.D.,0
14289,4492_88176,4492_452,O'Connell St Lower,0
14322,4492_88176,4492_455,Phoenix Pk,1


In [19]:
SelectDirection_df1["rout_dir_id_concat"] = (SelectDirection_df1["route_id"] + SelectDirection_df1["direction_id"].astype(str))
# Explicitly create a new DataFrame with only the required columns
SelectDirection_df1 = SelectDirection_df1[["trip_id", "route_id","direction_id", "rout_dir_id_concat"]].copy()

In [20]:
SelectDirection_df1

,trip_id,route_id,direction_id,rout_dir_id_concat
15,4486_11484,4486_88174,1,4486_881741
14,4486_11430,4486_88174,0,4486_881740
15978,4492_604,4486_88175,1,4486_881751
15566,4492_567,4486_88175,0,4486_881750
242,4486_563,4486_88176,1,4486_881761
...,...,...,...,...
10327,4492_19013,4486_88292,1,4486_882921
13078,4492_343,4492_88176,0,4492_881760
14289,4492_452,4492_88176,0,4492_881760
14322,4492_455,4492_88176,1,4492_881761


In [21]:
SelectDirection_df1.dtypes

trip_id               object
route_id              object
direction_id           int64
rout_dir_id_concat    object
dtype: object

In [22]:
stop_times_df1

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint
0,4486_10216,23:15:00,23:15:00,8240DB004843,1,Abbey St,0,1,1
1,4486_10216,23:15:47,23:15:47,8240DB004905,2,NaN,0,0,0
2,4486_10216,23:16:33,23:16:33,8240DB004906,3,NaN,0,0,0
3,4486_10216,23:17:06,23:17:06,8240DB004907,4,NaN,0,0,0
4,4486_10216,23:17:27,23:17:27,8240DB004908,5,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...
1076100,4492_9999,10:33:42,10:33:42,8240DB004912,56,NaN,0,0,0
1076101,4492_9999,10:34:24,10:34:24,8240DB004913,57,NaN,0,0,0
1076102,4492_9999,10:34:53,10:34:53,8240DB004914,58,NaN,0,0,0
1076103,4492_9999,10:35:29,10:35:29,8240DB004915,59,NaN,0,0,0


In [23]:
# Read the stop_times.txt file
file_path_4 = "stops.txt"  
stops_df = pd.read_csv(file_path_4)
stops_df["stop_full"] = (stops_df["stop_code"].astype(str) +' - '+stops_df["stop_name"])
stops_df.head(100)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_full
0,822008203,8203,Visitor Centre,NaN,53.364690,-6.331670,NaN,NaN,NaN,NaN,8203 - Visitor Centre
1,822008204,8204,Áras an Uachtaráin,NaN,53.359380,-6.323840,NaN,NaN,NaN,NaN,8204 - Áras an Uachtaráin
2,822008205,8205,Dublin Zoo,NaN,53.351730,-6.305480,NaN,NaN,NaN,NaN,8205 - Dublin Zoo
3,822008206,8206,Park Gates,NaN,53.348840,-6.297770,NaN,NaN,NaN,NaN,8206 - Park Gates
4,822008208,8208,Park Gates,NaN,53.348995,-6.298737,NaN,NaN,NaN,NaN,8208 - Park Gates
...,...,...,...,...,...,...,...,...,...,...,...
95,8220DB000127,127,Nursing Home,NaN,53.399108,-6.263680,NaN,NaN,NaN,NaN,127 - Nursing Home
96,8220DB000128,128,Glasnevin Avenue,NaN,53.389268,-6.265690,NaN,NaN,NaN,NaN,128 - Glasnevin Avenue
97,8220DB000129,129,Glasnevin Drive,NaN,53.389890,-6.271423,NaN,NaN,NaN,NaN,129 - Glasnevin Drive
98,8220DB000130,130,Glasnevin Park,NaN,53.390092,-6.274181,NaN,NaN,NaN,NaN,130 - Glasnevin Park


In [24]:
# Filter stop_times_df2 by trip_id in SelectDirection_df1
filtered_stop_times_df = stop_times_df1[stop_times_df1['trip_id'].isin(SelectDirection_df1['trip_id'])]

# Display the filtered DataFrame
filtered_stop_times_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint
0,4486_10216,23:15:00,23:15:00,8240DB004843,1,Abbey St,0,1,1
1,4486_10216,23:15:47,23:15:47,8240DB004905,2,NaN,0,0,0
2,4486_10216,23:16:33,23:16:33,8240DB004906,3,NaN,0,0,0
3,4486_10216,23:17:06,23:17:06,8240DB004907,4,NaN,0,0,0
4,4486_10216,23:17:27,23:17:27,8240DB004908,5,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...
1074675,4492_9975,07:43:36,07:43:36,8220DB000049,43,NaN,0,0,1
1074676,4492_9975,07:44:50,07:44:50,8220DB000051,44,NaN,0,0,0
1074677,4492_9975,07:45:30,07:45:30,8220DB000052,45,NaN,0,0,1
1074678,4492_9975,07:48:44,07:48:44,8220DB000277,46,NaN,0,0,1


In [25]:
StopByTrip_df=filtered_stop_times_df.copy()

In [26]:
# Explicitly create a new DataFrame with only the required columns
StopByTrip_df = StopByTrip_df[["trip_id", "stop_id","stop_sequence","stop_headsign","pickup_type","drop_off_type"]].copy()
# Renaming columns
StopByTrip_df.rename(columns={'pickup_type': 'trip_ends','drop_off_type': 'trip_starts'}, inplace=True)
StopByTrip_df

,trip_id,stop_id,stop_sequence,stop_headsign,trip_ends,trip_starts
0,4486_10216,8240DB004843,1,Abbey St,0,1
1,4486_10216,8240DB004905,2,NaN,0,0
2,4486_10216,8240DB004906,3,NaN,0,0
3,4486_10216,8240DB004907,4,NaN,0,0
4,4486_10216,8240DB004908,5,NaN,0,0
...,...,...,...,...,...,...
1074675,4492_9975,8220DB000049,43,NaN,0,0
1074676,4492_9975,8220DB000051,44,NaN,0,0
1074677,4492_9975,8220DB000052,45,NaN,0,0
1074678,4492_9975,8220DB000277,46,NaN,0,0


In [27]:
# Merge stop_times_df with the selected columns from stops_df on stop_id
StopByTrip_df = StopByTrip_df.merge(stops_df, on="stop_id", how="left")
StopByTrip_df = StopByTrip_df.merge(SelectDirection_df1, on="trip_id", how="left")
StopByTrip_df

,trip_id,stop_id,stop_sequence,stop_headsign,trip_ends,trip_starts,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_full,route_id,direction_id,rout_dir_id_concat
0,4486_10216,8240DB004843,1,Abbey St,0,1,4843,Berwick Walk,NaN,53.459664,-6.250397,NaN,NaN,NaN,NaN,4843 - Berwick Walk,4486_88211,1,4486_882111
1,4486_10216,8240DB004905,2,NaN,0,0,4905,Ormond Avenue,NaN,53.459964,-6.245144,NaN,NaN,NaN,NaN,4905 - Ormond Avenue,4486_88211,1,4486_882111
2,4486_10216,8240DB004906,3,NaN,0,0,4906,St Cronan's Avenue,NaN,53.460654,-6.241305,NaN,NaN,NaN,NaN,4906 - St Cronan's Avenue,4486_88211,1,4486_882111
3,4486_10216,8240DB004907,4,NaN,0,0,4907,St Cronan's Lawn,NaN,53.462539,-6.239917,NaN,NaN,NaN,NaN,4907 - St Cronan's Lawn,4486_88211,1,4486_882111
4,4486_10216,8240DB004908,5,NaN,0,0,4908,Ardcian Park,NaN,53.463944,-6.239512,NaN,NaN,NaN,NaN,4908 - Ardcian Park,4486_88211,1,4486_882111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13341,4492_9975,8220DB000049,43,NaN,0,0,49,Dorset Street Lower,NaN,53.360004,-6.260854,NaN,NaN,NaN,NaN,49 - Dorset Street Lower,4486_88210,1,4486_882101
13342,4492_9975,8220DB000051,44,NaN,0,0,51,Temple Street,NaN,53.357810,-6.263272,NaN,NaN,NaN,NaN,51 - Temple Street,4486_88210,1,4486_882101
13343,4492_9975,8220DB000052,45,NaN,0,0,52,Dorset Street Upper,NaN,53.356661,-6.264565,NaN,NaN,NaN,NaN,52 - Dorset Street Upper,4486_88210,1,4486_882101
13344,4492_9975,8220DB000277,46,NaN,0,0,277,O'Connell Street Upper,NaN,53.350822,-6.260506,NaN,NaN,NaN,NaN,277 - O'Connell Street Upper,4486_88210,1,4486_882101


In [28]:
StopByTrip_df.rename(columns={'rout_dir_id_concat': 'concat'}, inplace=True)

In [29]:
StopByTrip_df = StopByTrip_df[["route_id","trip_id","direction_id","concat","stop_id","stop_code", "stop_name","stop_full","stop_headsign","stop_sequence",
                              "stop_lat","stop_lon","trip_starts","trip_ends"]].copy()

In [30]:
StopByTrip_df

,route_id,trip_id,direction_id,concat,stop_id,stop_code,stop_name,stop_full,stop_headsign,stop_sequence,stop_lat,stop_lon,trip_starts,trip_ends
0,4486_88211,4486_10216,1,4486_882111,8240DB004843,4843,Berwick Walk,4843 - Berwick Walk,Abbey St,1,53.459664,-6.250397,1,0
1,4486_88211,4486_10216,1,4486_882111,8240DB004905,4905,Ormond Avenue,4905 - Ormond Avenue,NaN,2,53.459964,-6.245144,0,0
2,4486_88211,4486_10216,1,4486_882111,8240DB004906,4906,St Cronan's Avenue,4906 - St Cronan's Avenue,NaN,3,53.460654,-6.241305,0,0
3,4486_88211,4486_10216,1,4486_882111,8240DB004907,4907,St Cronan's Lawn,4907 - St Cronan's Lawn,NaN,4,53.462539,-6.239917,0,0
4,4486_88211,4486_10216,1,4486_882111,8240DB004908,4908,Ardcian Park,4908 - Ardcian Park,NaN,5,53.463944,-6.239512,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13341,4486_88210,4492_9975,1,4486_882101,8220DB000049,49,Dorset Street Lower,49 - Dorset Street Lower,NaN,43,53.360004,-6.260854,0,0
13342,4486_88210,4492_9975,1,4486_882101,8220DB000051,51,Temple Street,51 - Temple Street,NaN,44,53.357810,-6.263272,0,0
13343,4486_88210,4492_9975,1,4486_882101,8220DB000052,52,Dorset Street Upper,52 - Dorset Street Upper,NaN,45,53.356661,-6.264565,0,0
13344,4486_88210,4492_9975,1,4486_882101,8220DB000277,277,O'Connell Street Upper,277 - O'Connell Street Upper,NaN,46,53.350822,-6.260506,0,0


In [31]:
# Construct the full path for the output file
output_file_path4 = os.path.join(output_dir, "StopMapLocation.txt")

# Saving stop_times_per_route_df into StopTimesPerRoute.txt
StopByTrip_df.to_csv(output_file_path4, index=False) #Print this file into assets/data